In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras

tf.__version__

'1.15.0-dev20190820'

In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11493376/11490434 [==============================] - 2s 0us/step


In [5]:
# 短いシーケンシャルモデルを返す関数
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(784,)),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation=tf.keras.activations.softmax)
  ])

  model.compile(optimizer=tf.keras.optimizers.Adam(),
                loss=tf.keras.losses.sparse_categorical_crossentropy,
                metrics=['accuracy'])

  return model


# 基本的なモデルのインスタンスを作る
model = create_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [6]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# チェックポイントコールバックを作る
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model = create_model()

model.fit(train_images, train_labels,  epochs = 10,
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback])  # 訓練にコールバックを渡す


Train on 1000 samples, validate on 1000 samples
Epoch 1/10
  32/1000 [..............................] - ETA: 2s - loss: 2.2990 - acc: 0.1250
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 257us/sample - loss: 1.1395 - acc: 0.6800 - val_loss: 0.7110 - val_acc: 0.7850
Epoch 2/10
  32/1000 [..............................] - ETA: 0s - loss: 0.5924 - acc: 0.8125
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 91us/sample - loss: 0.4144 - acc: 0.8840 - val_loss: 0.5326 - val_acc: 0.8390
Epoch 3/10
  32/1000 [..............................] - ETA: 0s - loss: 0.3380 - acc: 0.9062
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 93us/sample - loss: 0.2752 - acc: 0.9350 - val_loss: 0.4644 - val_acc: 0.8480
Epoch 4/10
  32/1000 [..............................] - ETA: 0s - loss: 0.1151 - acc: 1.0000
Epoch 00004: saving model to training_1/cp.ckpt
1000/1000

In [7]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 47us/sample - loss: 2.3797 - acc: 0.1150
Untrained model, accuracy: 11.50%


In [8]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 20us/sample - loss: 0.3983 - acc: 0.8730
Restored model, accuracy: 87.30%


In [10]:
# ファイル名に(`str.format`を使って)エポック数を埋め込みます
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # 重みを5エポックごとに保存します
    period=5)

model = create_model()
model.fit(train_images, train_labels,
          epochs = 50, callbacks = [cp_callback],
          validation_data = (test_images,test_labels),
          verbose=0)

W0823 18:53:46.839293 4454114752 callbacks.py:864] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.



Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [11]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [12]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 56us/sample - loss: 0.4989 - acc: 0.8770
Restored model, accuracy: 87.70%


In [13]:
# 重みの保存
model.save_weights('./checkpoints/my_checkpoint')

# 重みの復元
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 59us/sample - loss: 0.4989 - acc: 0.8770
Restored model, accuracy: 87.70%


In [14]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

# モデル全体を１つのHDF5ファイルに保存します。
model.save('my_model.h5')

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 123us/sample - loss: 1.1699 - acc: 0.6590
Epoch 2/5
1000/1000 [==============================] - 0s 58us/sample - loss: 0.4347 - acc: 0.8720
Epoch 3/5
1000/1000 [==============================] - 0s 63us/sample - loss: 0.3001 - acc: 0.9140
Epoch 4/5
1000/1000 [==============================] - 0s 58us/sample - loss: 0.2109 - acc: 0.9480
Epoch 5/5
1000/1000 [==============================] - 0s 57us/sample - loss: 0.1543 - acc: 0.9660


In [16]:
# 重みとオプティマイザを含む全く同じモデルを再作成
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_8 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [17]:
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 77us/sample - loss: 0.4457 - acc: 0.8600
Restored model, accuracy: 86.00%


In [22]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 148us/sample - loss: 1.1325 - acc: 0.6890
Epoch 2/5
1000/1000 [==============================] - 0s 57us/sample - loss: 0.4129 - acc: 0.8880
Epoch 3/5
1000/1000 [==============================] - 0s 56us/sample - loss: 0.2937 - acc: 0.9170
Epoch 4/5
1000/1000 [==============================] - 0s 58us/sample - loss: 0.2070 - acc: 0.9450
Epoch 5/5
1000/1000 [==============================] - 0s 52us/sample - loss: 0.1443 - acc: 0.9740


In [26]:
saved_model_path = tf.contrib.saved_model.save_keras_model(model, "./saved_models")

W0826 08:31:22.810576 4454114752 export_utils.py:182] Export includes no default signature!
W0826 08:31:22.959507 4454114752 export_utils.py:182] Export includes no default signature!


In [27]:
new_model = tf.contrib.saved_model.load_keras_model(saved_model_path)
new_model

W0826 08:31:32.371666 4454114752 util.py:144] Unresolved object in checkpoint: (root).optimizer.beta_1
W0826 08:31:32.372447 4454114752 util.py:144] Unresolved object in checkpoint: (root).optimizer.beta_2
W0826 08:31:32.373094 4454114752 util.py:144] Unresolved object in checkpoint: (root).optimizer.decay
W0826 08:31:32.373569 4454114752 util.py:144] Unresolved object in checkpoint: (root).optimizer.learning_rate
W0826 08:31:32.374319 4454114752 util.py:144] Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).layer_with_weights-0.kernel
W0826 08:31:32.375118 4454114752 util.py:144] Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).layer_with_weights-0.bias
W0826 08:31:32.375756 4454114752 util.py:144] Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).layer_with_weights-1.kernel
W0826 08:31:32.376435 4454114752 util.py:144] Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).layer_with_weights-1.bia

TypeError: Expected binary or unicode string, got None

In [ ]:
# バージョンの問題とかで保存できないっぽい。実験的な機能って注釈付きなのでいったん以降は無視。